### build-mysql-database
CONSTROI BASE DE DADOS NO MYSQL POR MEIO DE DOCUMENTOS DA COLEÇÃO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS INFORMAÇÕES DE CADA DOCUMENTO A SEREM INSERIDAS NA BASE SÃO:
* ID
* CORPO DE TEXTO
* DATAS (ANO E MÊS)
* URL
* IDIOMA PRINCIPAL

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import pymysql
import getpass

from sshtunnel import SSHTunnelForwarder
import time

import codecs
from langdetect import detect
from IPython.display import clear_output
import csv
import xlrd
import xlwt

In [18]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria função que lista arquivos de forma ordenada
Precisei atentar para arquivos com numeração distinta, tais como:
AAS_mre_onu_1975.01.23_doc_I-A.txt
AAS_mre_onu_1975.01.23_doc_I-6A8.txt
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
def to_zero(x):
    if x == '': x = '0'
    return x
def special_sort(l):
    convert = lambda text: int(text) if text.isdigit() else str(text)
    alphanum_key = lambda key: [ convert(to_zero(c)) for c in filter(None, re.split('(\d)A|A\d|([A-Z]*)-A?|.txt', key))] 
    return sorted(l, key = alphanum_key)

In [19]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

path = "C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/"
#path = "C:/Users/marcelo.ribeiro/Documents/temp/" #TESTE!
#outputs = "C:/Users/marcelo.ribeiro/Documents/outputs/"
#path = "/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/"

files = [f for f in sorted(os.listdir(path))]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)
fullpath_list = special_sort(fullpath_list)

In [20]:
fullpath_list[0:10]

['C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-1.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-2.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-3.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-4.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-5.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-6.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-7.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-8.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mre_ag_1973.11.20_doc_I-9.txt',
 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/AAS_mr

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria listas de url e dossies
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
url_list = []
dossie_url_list = []
url_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/URLS-AAS-marcelo.csv"
#url_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/URLS-AAS_2.csv"
with open(url_inputs, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if ' mre ' not in row[1]: continue
        dossie = re.sub(' ','_',row[1])
        dossie = re.sub('AAS_mre_(.*)',r'\1',dossie)
        dossie = re.sub('\/',r'-',dossie)
        dossie_url_list.append(dossie)
        url_list.append(row[2])
        

In [14]:
#print(dossie_url_list)
dossie = dossie_url_list[0]
dossie_date = re.sub('.*(19\d\d\.\d\d\.\d\d).*', r'\1', dossie)
dossie_year = re.sub('.*(19\d\d).*', r'\1', dossie)
dossie_month = re.sub('.*19\d\d\.(\d\d).*', r'\1', dossie)
dossie_id = dossie
print(dossie)
print(dossie_date )
print(dossie_year )
print(dossie_month)
print(dossie_id)

d_1974.03.26
1974.03.26
1974
03
d_1974.03.26


In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria lista de descritores
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
dossie_sub_list = []
subject_list  = []
keywords_list = []

path_inputs = "../inputs/"
workbook = xlrd.open_workbook(path_inputs+'AAS-mre-metadados_Accessus.xlsx', on_demand = True)
worksheet = workbook.sheet_by_index(0)

subject_list = []
keywords_list = []

for line in range(1, worksheet.nrows):
#for line in range(1, 3):
    if worksheet.cell(line, 0).value == '': break
    subject = str(worksheet.cell(line, 4).value)
    keywords = str(worksheet.cell(line, 10).value)
    subserie = worksheet.cell(line, 3).value
    subserie = re.sub('(\w)\w*( do p?r?e?s?i?d?e?n?t?e? ?| com o |[\s-])(\w)\w*',r'\1\3',subserie)
    subserie = subserie.lower()
    if subserie == 'dp geisel': subserie = 'd'
    if subserie == 'vg': subserie = 'vp'
    dossie_date = str(worksheet.cell(line, 1).value)
    dossie_date = re.sub('\/',r'-',dossie_date)
    dossie = subserie+'_'+dossie_date
    dossie_sub_list.append(dossie)
    subject_list.append(subject)
    keywords_list.append(keywords)


In [4]:
print(len(dossie_sub_list))
print(len(subject_list))
print(len(keywords_list))
print(len(url_list))
print(len(dossie_url_list))

76
76
76
76
76


In [22]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria e alimenta a base mysql
Classifica o corpus de acordo com: tipo textual (para separar textos sujos, como letra cursiva, foto, fonte alternativa) e idioma
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
temp = 0
doc_class = []
count_doc = 0

### adiciona info não-variável para as colunas collection e is_date_exact ###
is_date_exact = 0 # datas não são precisas
collection = "CPDOC"

### inicia o SQL ###
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS dossies")
    cur.execute('''CREATE TABLE dossies 
               (id VARCHAR(31) PRIMARY KEY, subject LONGTEXT, keywords LONGTEXT, date DATETIME, date_year SMALLINT(6), date_month TINYINT(4), is_date_exact TINYINT(1), 
               collection VARCHAR(16), url VARCHAR(200)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    for dossie in dossie_url_list:
        
        ### captura info de data, ano, mes e id de cada dossie ###
        dossie_id = re.sub('.*AAS_mre_(.*).txt', r'\1', dossie)
        dossie_date = re.sub('.*(19\d\d\.\d\d\.\d\d).*', r'\1', dossie)
        dossie_year = re.sub('.*(19\d\d).*', r'\1', dossie)
        dossie_month = re.sub('.*19\d\d\.(\d\d).*', r'\1', dossie)
        
        ### captura info de url ###
        list_indexnumber = dossie_url_list.index(dossie)
        url = url_list[list_indexnumber]
       
        ### captura info de descritores ###
        for i in dossie_sub_list:
            if i == dossie:
                list_indexnumber = dossie_sub_list.index(i)
                subject = subject_list[list_indexnumber]
                keywords = keywords_list[list_indexnumber]
            
        ### insere dados na base mysql ###
        query = "INSERT INTO dossies VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(query, (dossie_id, subject, keywords, dossie_date, dossie_year, dossie_month, is_date_exact, collection, url))

········
········
